Introduction, QR Code Scanner App
===
Using ZXing (Zebra Crossing) Scanner Library
---
**Some facts about Quick Response(QR) code**
- a two dimensional barcode(matrix codes) that allows content to be decoded at a high speed;
- invented in 1994 by japanese company Denso-Wave.
- license-free,
- consists of black modules (square dots) arranged in a square grid on a white background, readable by an imaging device (such as a camera, scanner, etc.)
- dimensions, 21×21 pixel size is version 1, 25×25 is version 2, and so on. The 177×177 size is version 40.


In [3]:
import pyqrcode
url = pyqrcode.create('What a wonderful world!')
#url.svg('url.svg', scale=8)
#url.eps('url.eps', scale=2)
url.png('data/qrblock.png',scale=4)
print(url.terminal(quiet_zone=1))

Generated QR block
---
<img src="data/qrblock.png"  width="50%" />

In [7]:
import io
import base64
from IPython.display import HTML

video = io.open('data/QR.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

Subject
---
Using **ZXing Library** to  implement the process of scanning the image of the QR Code on the click of Button.

Steps
---
1. implement the **ZXingScannerView.ResultHandler** class to handle the scanned result in the **MainActivity.java** file 
- initialize the **ZXingScannerView** in **MainActivity.java** file. It will start your camera and scan the image of QR Code to decode the QR code. 
- After complete scanning of QR Code, result is handled by **handleResult()** method.



Demo Project
---
- Create a new Project with Empty Activity, 
<code>
    Project: QrCodeScannerProject
    company domain: com.qrcode.scanner
</code>
in Android Studio, goto ```[Start a New Project]```, .

1. AndroifManifest.xml
---
Add permisions as follows:

```XML
<?xml version="1.0" encoding="utf-8"?>
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    package="qrcodescanner.com.qrcodescannerproject">
    <uses-permission android:name="android.permission.CAMERA" />
    <uses-permission android:name="android.permission.WRITE_EXTERNAL_STORAGE" />
    <uses-permission android:name="android.permission.INTERNET" />

    <uses-feature android:name="android.hardware.Camera" />
    <uses-feature android:name= "android.hardware.camera.autofocus " />

    <application
        android:allowBackup="true"
        android:icon="@mipmap/ic_launcher"
        android:label="@string/app_name"
        android:roundIcon="@mipmap/ic_launcher_round"
        android:supportsRtl="true"
        android:theme="@style/AppTheme">
        <activity android:name=".MainActivity">
            <intent-filter>
                <action android:name="android.intent.action.MAIN" />

                <category android:name="android.intent.category.LAUNCHER" />
            </intent-filter>
        </activity>
    </application>

</manifest>
```


2. Adding Library in the dependencies:
---
To compile the **ZXing** library, we need to add the library in the app’s dependencies. Open the app’s ```build.gradle (Module app)``` file. Add the library in the dependencies. It will compile the library at run time.

```XML
...
dependencies {
    compile fileTree(dir: 'libs', include: ['*.jar'])
    androidTestCompile('com.android.support.test.espresso:espresso-core:2.2.2', {
        exclude group: 'com.android.support', module: 'support-annotations'
    })
    compile 'com.journeyapps:zxing-android-embedded:2.0.1@aar'
    compile 'com.journeyapps:zxing-android-legacy:2.0.1@aar'
    compile 'com.journeyapps:zxing-android-integration:2.0.1@aar'
    compile 'com.google.zxing:core:3.0.1'

    compile 'com.android.support:appcompat-v7:25.3.1'
    compile 'com.android.support.constraint:constraint-layout:1.0.0-beta4'
    testCompile 'junit:junit:4.12'
}
```
Click [sync] to let  Android studio automatically download the **zxing** libraries we added.




3. Layout
---
change the Layout style to RelativeLayout and add one button and one one TextView:

```HTML
<?xml version="1.0" encoding="utf-8"?>
<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context="qrcodescanner.com.qrcodescannerproject.MainActivity">

    <Button
        android:id="@+id/button1"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:onClick="QrScanner"
        android:text="QR Scanner"
        tools:layout_editor_absoluteX="8dp"
        tools:layout_editor_absoluteY="0dp" />

    <TextView
        android:id="@+id/textview1"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_below="@id/button1"
        android:textColor="#f00"
        android:textSize="16px"
        android:text="Press Button  to get informance." />
</RelativeLayout>        

```

4. **MaimActivity.java**
---
**Processing Flow**

<pre style="color:brown;fontsize:1.4em;font-family:Chalkboard;">
       (1)                                   (2)
   Click button    --->    activate Camera   --->  focus the QR or BarCode
                                                          |
                                                          |  (3)
                                                          |
                                                         ▼
                response       < ------      return the Info's      
</pre>  
   
1. Waiting for button clicked
```java
button.setOnClickListener(new View.OnClickListener() { 
                 // clicked  
              });
```
- start Scanning
```java
               public void onClick(View v) {
                  IntentIntegrator scanIntegrator = new IntentIntegrator(mainactivity);
                  scanIntegrator.initiateScan();
               }
```
- return info's
```java
   public void onActivityResult(int requestCode, int resultCode, Intent intent){
        IntentResult scanningResult = IntentIntegrator.parseActivityResult(requestCode, resultCode, intent);

        if(scanningResult!=null){
            // if something
        }else{
            // if nothing
        }
    }
```    

Main part of Code, MainActivity.java
---
1. import necessary packages and change the Activity,
- init Scan processing, ```init_view()```,
- set up the button function, ```button.setOnClickListener()```,
- Scan, ```IntentIntegrator(mainactivity)```,
- return the result, ```onActivityResult()```. Here display the content:

```java
...
package qrcodescanner.com.qrcodescannerproject;

import android.app.Activity;
import com.google.zxing.integration.android.*;
import android.content.Intent;
import android.os.Bundle;
import android.view.View;
import android.widget.Button;
import android.widget.TextView;
import android.net.Uri;
import android.widget.Toast;

public class MainActivity extends Activity{

    private Activity mainactivity;
    private TextView scan_content;
    private Button scan_btn;
    
    @Override
    protected void onCreate(Bundle saveInstanceStat){
        super.onCreate(saveInstanceStat);

        setContentView(R.layout.activity_main);
        init_view();
    
        /* Wait for service by clicking button */ 
        scan_btn.setOnClickListener(new View.OnClickListener() {
            @Override
            public void onClick(View v) {
                IntentIntegrator scanIntegrator = new IntentIntegrator(mainactivity);
                // Change the camera from front one to back one
                // scanIntegrator.addExtra("SCAN_MODE", "QR_CODE_MODE");
                // scanIntegrator.addExtra("SCAN_CAMERA_ID", 1);
                scanIntegrator.initiateScan();
            }
        }); 
    }
    
    public void onActivityResult(int requestCode, int resultCode, Intent intent){
        IntentResult scanningResult = IntentIntegrator.parseActivityResult(requestCode, resultCode, intent);

        if(scanningResult!=null){
            String scanContent=scanningResult.getContents();
            scan_content.setText(scanContent);
        }else{
            Toast.makeText(getApplicationContext(),"nothing",Toast.LENGTH_SHORT).show();
        }
    }
    
    private void init_view(){
        this.scan_content=(TextView)findViewById(R.id.textview1);
        this.mainactivity=this;
        this.scan_btn = (Button)findViewById(R.id.button1);
    }
}
```